In [7]:
# see https://nitratine.net/blog/post/python-gcm-encryption-tutorial/

from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES
from Crypto.Protocol.KDF import scrypt

BUFFER_SIZE = 1024 * 1024  # The size in bytes that we read, encrypt and write to at once

password = "password"  # Get this from somewhere else like input()

input_filename = 't.txt'  # Any file extension will work
output_filename = input_filename + '.encrypted'  # You can name this anything, I'm just putting .encrypted on the end


def write_header(file_out, salt, cipher):
    file_out.write(salt)
    file_out.write(cipher.nonce)


def write_body(file_in, file_out, cipher):
    while len(data := file_in.read(BUFFER_SIZE)):
        encrypted_data = cipher.encrypt(data)  # Encrypt the data we read
        file_out.write(encrypted_data)  # Write the encrypted data to the output file


def write_footer(cipher, file_out):
    tag = cipher.digest()  # Signal to the cipher that we are done and get the tag
    file_out.write(tag)


def generate_crypto_values(password):
    salt = get_random_bytes(32)
    key = scrypt(password, salt, key_len=32, N=2 ** 17, r=8, p=1)
    cipher = AES.new(key, AES.MODE_GCM)
    return salt, key, cipher

with open(output_filename, 'wb') as  file_out:
    salt, key, cipher = generate_crypto_values(password=password)
    write_header(file_out=file_out, salt=salt, cipher=cipher)
    with open(input_filename, 'rb') as  file_in:
        write_body(file_in=file_in, file_out=file_out, cipher=cipher)
    write_footer(cipher=cipher, file_out=file_out)
